In [1]:
import numpy as np

%matplotlib ipympl
import matplotlib.pyplot as plt

I don't understand why you need to run this command twice to get an interactive matplotlib display

In [2]:
%matplotlib ipympl

In [3]:
import lsst.daf.persistence        as dafPersist
import lsst.pex.exceptions         as pexExcept

import lsst.afw.display            as afwDisplay
import lsst.daf.base               as dafBase

import lsst.afw.image              as afwImage
import lsst.afw.geom               as afwGeom
import lsst.afw.table              as afwTable

import lsst.meas.algorithms        as measAlg

#### Load the high-level "tasks" that process the pixels

In [4]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

#### Setup the displays (by default an interface to ds9)

In [ ]:
afwDisplay.setDefaultBackend("matplotlib")

## Create the tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask =         CharacterizeImageTask(None, config=config)

sourceDetectionTask =   SourceDetectionTask(schema=schema)

sourceDeblendTask =     SourceDeblendTask(schema=schema)

if False:
    sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                       algMetadata=algMetadata)
else:
    config = SingleFrameMeasurementTask.ConfigClass()
    config.slots.apFlux = 'base_CircularApertureFlux_12_0'
    
    if True:    # Warning:  this takes too long -- be prepared to wait when running sourceMeasurementTask
        import lsst.meas.modelfit
        config.plugins.names |= ["modelfit_DoubleShapeletPsfApprox", "modelfit_CModel"]
        config.slots.modelFlux = 'modelfit_CModel'
        #config.catalogCalculation.plugins['base_ClassificationExtendedness'].fluxRatio = 0.985
        
    sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                       algMetadata=algMetadata)

## Time to process some data

#### Read the input data

In [ ]:
butler = dafPersist.Butler("/datasets/hsc/repo/rerun/RC/w_2019_18/DM-19151")

In [ ]:
dataId = dict(visit=1232, ccd=51)
exposure = butler.get('calexp', dataId)

In [ ]:
disp = afwDisplay.Display(reopenPlot=True, fastMaskDisplay=True)

disp.scale('asinh', -20, 30, Q=8)
disp.mtv(exposure, title=dataId)

disp.zoom(8, 500, 1500);

FigureCanvasNbAgg()

#### Create the output table

In [ ]:
tab = afwTable.SourceTable.make(schema)

### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [ ]:
result = charImageTask.run(exposure)

##### Detect objects (`sources')

In [ ]:
result = sourceDetectionTask.run(tab, exposure)
sources = result.sources

##### Deblend overlapping objects

In [ ]:
sourceDeblendTask.run(exposure, sources)

##### Measure the objects' properties

In [ ]:
sourceMeasurementTask.run(sources, exposure)

#### Write the results to a FITS file (if desired)

In [ ]:
if False:
    fitsTableFile = "outputTable.fits"
    sources.writeFits(fitsTableFile)

    exposure.writeFits("example1-out.fits")
else:
    fitsTableFile = None

## OK, we've finished the image processing.

We have our list of sources, but it's not currently continuous in memory

In [ ]:
try:
    sources.get("id")
except Exception as e:
    print(e)

We can fix this with a deep copy

In [ ]:
sources = sources.copy(True)

In [ ]:
if fitsTableFile is not None:
    sources = afwTable.SourceCatalog.readFits(fitsTableFile)

## Look at the results

In [ ]:
import lsst.afw.display.utils as afwDisplayUtils

Define a boolean array that tells us which objects are 'good'; in this case:
 - No saturated pixels near their centres
 - terminal objects that haven't been further deblended

In [ ]:
good = np.logical_and.reduce([sources.get('base_PixelFlags_flag_interpolatedCenter') == 0,
                              sources.get("deblend_nChild") == 0,
                              ])

#### Look at how well the aperture photometry agrees with the PSF measurements

In [ ]:
if False:
    apMag = exposure.getPhotoCalib().instFluxToMagnitude(sources, 'base_CircularApertureFlux_12_0')
else:
    apMag = exposure.getPhotoCalib().instFluxToMagnitude(sources, 'slot_ApFlux')
  
try:
    modelMag = exposure.getPhotoCalib().instFluxToMagnitude(sources, 'modelfit_CModel')
except pexExcept.NotFoundError:
    modelMag = None

psfMag = exposure.getPhotoCalib().instFluxToMagnitude(sources, 'slot_PsfFlux')

if False or modelMag is None:
    mag1, str1 = apMag, "ap"
else:
    mag1, str1 = modelMag, "model"
mag2, str2 = psfMag, "psf"

plt.close(1); plt.figure(1)
plt.plot(mag1[good], (mag2 - mag1)[good], 'o')

plt.axhline(0.0, ls=':', color='black')
plt.xlim(14, 24.5)
plt.ylim(-0.2, 0.8)

plt.xlabel(str1)
plt.ylabel(f"{str2} - {str1}")

plt.show()

This doesn't look like modern CCD data as there's no brighter-fatter effect.  That's because we corrected it at the pixel level as part of the instrumental signature effect.

## What photometry is available?

In [ ]:
for k in sources.getSchema().extract("*_instFlux").keys():
    if "slot" not in k and "Blendedness" not in k:
        print(k)

In [ ]:
disp2 = afwDisplay.Display(2, reopenPlot=True)

disp2.setMaskPlaneColor("CROSSTALK", afwDisplay.IGNORE)   # I don't care about crosstalk-corrected pixels

disp2.scale('asinh', 'zscale', Q=4)
disp2.mtv(exposure, title='post calib')

##### Overlay something about the measurements

In [ ]:
disp2.erase()
with disp2.Buffering():
    for s in sources[good]:
        disp2.dot('+', *s.getCentroid(),
                  ctype=afwDisplay.GREEN if s.get("parent") == 0 else afwDisplay.CYAN)

In [ ]:
if True:
    radii = algMetadata.getArray("base_CircularApertureFlux_radii")

    with disp2.Buffering():
        for s in sources[good]:
            if exposure.getPhotoCalib().instFluxToMagnitude(s, 'slot_ApFlux').value < 20.0:
                for radius in radii:
                    disp2.dot('o', *s.getCentroid(), size=radius, ctype=afwDisplay.YELLOW)

### Astropy

In [ ]:
sources[good].copy(deep=True).asAstropy()